# Responses to MusicLab via Nettskjema API

In [22]:
import requests
import json
import zipfile
import socket
import os
import base64
import time
import pandas as pd

navigate to the directory storing your *nettskjema_token.txt* file

In [3]:
os.listdir()

['PullingMusicLab.ipynb',
 '.DS_Store',
 'PullingNettskjema.ipynb',
 'README.md',
 '.gitignore',
 'copen_netts.tsv',
 'Test_API',
 '.ipynb_checkpoints',
 '.git',
 'nettskjema_token.txt']

In [4]:
f=open('nettskjema_token.txt','r')
TOKEN = f.read()
f.close()

In [5]:
session = requests.Session()
session.headers.update({'Authorization': 'Bearer ' + TOKEN})

Test the token by sending a request to see its expiration date.

In [6]:
# confirm the token is working on this IP with the check on expiry date
request_url = "https://nettskjema.no/api/v2/users/admin/tokens/expire-date"
response = session.get(request_url)
response.content

b'{"expireDate":"2022-10-18T17:38:29.000+0200"}'

# Gathering Musiclab phone sensor data


In [7]:
# get metadata on submissions for the music lab app

formID = 141510
submissionID = 16561653
request_url = 'https://nettskjema.no/api/v2/forms/' + str(formID) + '/submissions?fromSubmissionId=' + str(submissionID)
response = session.get(request_url)
subIDs = json.loads(response.content.decode())
print('Submissions since subID ' + str(submissionID) + ': ' + str(len(subIDs)))
if len(subIDs)<5: # print error message or top responses 
    print(subIDs)
else:
    print(subIDs[0])


Submissions since subID 16561653: 23503
{'submissionId': 16767882, 'createdDate': '2021-10-29T08:40:57.000+0200', 'modifiedDate': '2021-10-29T08:40:57.000+0200', 'delivered': True, 'answerTime': 0, 'answers': [{'answerId': 103133021, 'questionId': 1996787, 'textAnswer': 'cfcd73d7-4af9-08e2-aa6a-e73e87bbfbf3'}, {'answerId': 103133022, 'questionId': 1996788, 'textAnswer': 'data.zip', 'attachments': [{'answerAttachmentId': 302365, 'fileName': 'data.zip', 'mediaType': 'application/zip'}]}]}


test one submission data

In [13]:
subn = 0
print('submissionID : ' + str(subIDs[subn]['submissionId']))
for subm in subIDs[subn]['answers']:
    if len(subm)<4:
        devID = subm['textAnswer']
        print('Submitting instillation: ' + devID)
    else:
        att_dets = subm['attachments'][0] # the app only attaches one zipped file called data.zip
        print(att_dets)

submissionID : 16767882
Submitting instillation: cfcd73d7-4af9-08e2-aa6a-e73e87bbfbf3
{'answerAttachmentId': 302365, 'fileName': 'data.zip', 'mediaType': 'application/zip'}


so to call the attachment for that submission we use the request:

In [14]:
subn = 0 # just calling one as an example

subID = str(subIDs[subn]['submissionId'])
attID = str(att_dets['answerAttachmentId'])
request_url = 'https://nettskjema.no/api/v2/submissions/' + subID + '/attachments/' + attID
response = session.get(request_url)

att_dets = json.loads(response.content.decode())
print(att_dets)


{'fileName': 'data.zip', 'fileSize': 10637, 'mediaType': 'application/zip', 'content': 'UEsDBAoAAAAAABw1XVOfPYrJwSgAAMEoAAA1AAAAY2ZjZDczZDctNGFmOS0wOGUyLWFhNmEtZTczZTg3YmJmYmYzLmRldmljZU1vdGlvbi5jc3Z0aW1lc3RhbXAsdGltZSx4LHkseixhbHBoYSxiZXRhLGdhbW1hDQoxNjM1NDg5NjUyNTAyLDk2OTcxLDAuODkzOCw3LjE0MSw2Ljg4MDksLTUuMTAxOSwyLjcwNTIsMC4zNTU0DQoxNjM1NDg5NjUyNTE1LDk2OTgzLDAuMTg3Niw3LjY4MDksNi40MzU5LDIuNzA3OCwtMi40MjE2LDAuMTI3OA0KMTYzNTQ4OTY1MjUzMiw5NzAwMCwwLjE1MjgsNy43MDExLDYuMzMxNiw0LjI4NTUsLTAuNjIwNSwwLjc4NTcNCjE2MzU0ODk2NTI1NDgsOTcwMTcsMC4xMTIxLDcuNjcyMiw2LjI5NDQsNS40MzY5LDEuNzEyMywxLjQ5NDMNCjE2MzU0ODk2NTI1NjUsOTcwMzMsMC4wOTU5LDcuNjYzNSw2LjI4MTYsNi4yMjkyLDIuOTI0NiwxLjcNCjE2MzU0ODk2NTI1ODIsOTcwNTAsMC4xODc1LDcuNjU4NSw2LjE5OTksNS44ODA1LDIuNzY0NiwxLjUwMDgNCjE2MzU0ODk2NTI1OTksOTcwNjcsMC4yNTY5LDcuNjc3OCw2LjExNTgsNC45NjUxLDIuMzU4NiwwLjk3NjcNCjE2MzU0ODk2NTI2MTUsOTcwODMsMC4yNzMxLDcuNjkyOSw2LjA5MTQsNC4zNDY0LDEuODA5OCwwLjI0MTkNCjE2MzU0ODk2NTI2MzIsOTcxMDAuMDAwMDAwMDAwMDEsMC4yMjg2LDcuNzI0Myw2LjAyMTYsMy4yMzE0

The nettskjema API returns attachments as encoded zipfiles in byte strings. To make these readable, we need to decode then save the string as a zip file and then unzip them. Thankfully there are python libraries for this.  

First be sure you are in a suitable local folder, then unpack the attachement.

In [15]:
os.mkdir('./Copendata')
os.chdir('Copendata')

In [69]:
subn = 0 # just calling one as an example

subID = str(subIDs[subn]['submissionId'])
attID = str(subIDs[subn]['answers'][0]['attachments'][0]['answerAttachmentId'])
request_url = 'https://nettskjema.no/api/v2/submissions/' + subID + '/attachments/' + attID
response = session.get(request_url)

att_dets = json.loads(response.content.decode())

# write the decoded attachment into a zip file
f=open('data.zip', 'wb')
f.write(base64.b64decode(att_dets['content']))
f.close()

# and then unzip that file, leaving a uniquely titled csv, I hope
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    if not os.path.exists(str(subID)):
        os.mkdir(str(subID))
        zip_ref.extractall('./'+str(subID)) # Not unique filenames so use the unique submission IDs 

print(os.listdir())
os.chdir('./'+str(subID))
print(os.listdir())
os.chdir('..')

['data.zip', '16767882']
['cfcd73d7-4af9-08e2-aa6a-e73e87bbfbf3.deviceMotion.csv']


Here we have a minute recording from a device with the unique installation ID 'cfcd73d7-4a...' in a format that is easy to read. 

The files within the zip are named for the device and information type, but do not include the submission number. This means if they are opened into a folder that already contains a previous recording from the same device, the previously saved recording will be overwritten and lost. To avoid this, the files are unzipped within a folder names for that unique submission. 

Now to collect many at once: 

In [82]:
# pull in the data files and unpack them

newSubs = 0 # count the submission sampled
tic = time.time()
for submis in subIDs[:10]: # just getting 10 as a test
    # first find out the attachment file ID for this submission
    subID = str(submis['submissionId'])
    # if there is an IDed attachment for this submission, get the file
    for subm in submis['answers']:
        if len(subm)>3:
            attID = str(subm['attachments'][0]['answerAttachmentId'])
            request_url = 'https://nettskjema.no/api/v2/submissions/' + subID + '/attachments/' + attID
            response = session.get(request_url)
            newSubs += 1
            att_dets = json.loads(response.content.decode())
            # write the decoded attachment into a zip file
            f=open('data.zip', 'wb')
            f.write(base64.b64decode(att_dets['content']))
            f.close()
            # and then unzip that file, leaving a uniquely titled csv, I hope
            with zipfile.ZipFile('data.zip', 'r') as zip_ref:
                if not os.path.exists(str(subID)):
                    os.mkdir(str(subID))
                    zip_ref.extractall('./'+str(subID)) # if not unique can use the unique submission IDs 

print('time to collect ' + str(newSubs) + ' attachments: ' + str(time.time() - tic))

time to collect 10 attachments: 0.8018779754638672


In [9]:
#os.chdir('nettskjemaData')
os.chdir('CopenData')

In [28]:
subs_already  = os.listdir()#.sort()
subs_already.sort()
print(len(subs_already))
lastsub = '16609378' #subs_already[-2]
lastsub 
# 16550625

2274


'16609378'

In [10]:
# collect submission IDs to form from a specific date
#form_ID = 141510
#submissions_ID_range = 'https://nettskjema.no/api/v2/forms/' + str(form_ID) + '/submissions?fields=submissionId&fromSubmissionId=' + lastsub
submissions_ID_range = 'https://nettskjema.no/api/v2/forms/141510/submissions?fields=submissionId&fromDate=2021-10-25T13%3A43%3A17.486%2B0100'

response = session.get(submissions_ID_range)
subIDs = json.loads(response.content.decode())#eval(response.content.decode())
print(len(subIDs))
if len(subIDs)<5:
    print(subIDs)
else:
    print(subIDs[0])
    
#1976
# {'submissionId': 16609378}

354
{'submissionId': 16638490}


In [17]:
# pull in all the data as fast as possible.
submission_dets = 'https://nettskjema.no/api/v2/submissions/'
newSubs = 0
tic = time.time()
for subm in subIDs[1000:]:
    # first find out the attachment file ID for this submission
    subid = subm["submissionId"]
    response = session.get(submission_dets + str(subid) + '/attachments')
    sub_atts = json.loads(response.content.decode())
    # if there is an IDed attachment for this submission, get the file
    if len(sub_atts)>0:
        attid = sub_atts[0]
        newSubs += 1
        response = session.get(submission_dets + str(subid) + '/attachments/' + str(attid))
        att_dets = json.loads(response.content.decode())
        # write the decoded attachment into a zip file
        f=open('data.zip', 'wb')
        f.write(base64.b64decode(att_dets['content']))
        f.close()
        # and then unzip that file, leaving a uniquely titled csv, I hope
        with zipfile.ZipFile('data.zip', 'r') as zip_ref:
            if not os.path.exists(str(subid)):
                os.mkdir(str(subid))
                zip_ref.extractall('./'+str(subid)) # if not unique can use the unique submission IDs 
print(time.time() - tic)
print(newSubs)

4047.5774102211
22503



for subid in folders:
    if subid.startswith('1'):
        filenames = os.listdir('./CopenData/'+str(subid))
        print(filenames)
        for fn in filenames:
            if fn.endswith('.csv'):
                print(fn)
                
                sourcefile = './CopenData/'+str(subid) + '/' + fn
                targetfile = './CompressedData/'+str(subid) + '.' + fn
                print(targetfile)
                os.rename(sourcefile,targetfile)

In [ ]:
# one option is to pack a batch into a single folder, 
# files renamed with the combo of installation number and submission ID
# 

# Check out other forms

In [19]:
formes = [141510,225781,225357,225336,225387,225707,225692,225713,225713,225702,225709,225693,225714,225695,225711]


In [25]:
formID = formes[4]
request_url = 'https://nettskjema.no/api/v2/forms/' + str(formID)
response = session.get(request_url) # using the request session call which includes the saved API token
form_metadata = json.loads(response.content.decode()) # inteprete recieved string into a python native datatype
form_metadata # show the information output

{'formId': 225387,
 'languageCode': 'en',
 'title': '2 After Beethoven (English) MusicLab Copenhagen ',
 'deliveryDestination': 'DATABASE',
 'formType': 'DEFAULT',
 'theme': 'DEFAULT',
 'createdBy': {'personId': 616293,
  'username': 'danasw@uio.no',
  'fullName': 'Dana Swarbrick',
  'name': 'Dana Swarbrick',
  'type': 'LOCAL'},
 'modifiedBy': {'personId': 1676908,
  'username': 'finnu@uio.no',
  'fullName': 'Finn Upham',
  'name': 'Finn Upham',
  'type': 'LOCAL'},
 'createdDate': '2021-10-20T10:27:02.000+0200',
 'modifiedDate': '2021-10-29T09:35:14.000+0200',
 'openFrom': '2021-10-20T17:40:44.000+0200',
 'respondentGroup': 'ALL',
 'editorsContactEmail': 'danasw@uio.no',
 'editorsSubmissionEmailType': 'NONE',
 'editors': [{'personId': 573969,
   'username': 'kaylab@uio.no',
   'fullName': 'Kayla Burnim',
   'name': 'Kayla Burnim',
   'type': 'LOCAL'},
  {'personId': 1914058,
   'username': 'finnu1@api',
   'fullName': 'finn ritmo access',
   'name': 'finn ritmo access',
   'type': 'API

In [26]:
elem = form_metadata['elements']

for m in elem:
    print(str(m['elementId']) + ' ' + m['elementType'])
    if 'questions' in m: 
        for q in m['questions']:
            print('\t Qid: ' + str(q['questionId']))
            print('\t Q: ' + str(q['text']))
            if 'answerOptions' in q:
                for a in q['answerOptions']:
                    print('\t \t' + a['text'])

3484547 QUESTION
	 Qid: 3740343
	 Q: userID
3479334 TEXT
3479777 RADIO
	 Qid: 3734711
	 Q: Tears.
	 	0 (Not at all)
	 	1
	 	2
	 	3
	 	4
	 	5
	 	6 (a lot)
3479778 RADIO
	 Qid: 3734712
	 Q: Chills or shivers.
	 	0 (Not at all)
	 	1
	 	2
	 	3
	 	4
	 	5
	 	6 (a lot)
3479779 RADIO
	 Qid: 3734713
	 Q: A warm feeling in the center of the chest.
	 	0 (Not at all)
	 	1
	 	2
	 	3
	 	4
	 	5
	 	6 (a lot)
3479780 RADIO
	 Qid: 3734714
	 Q: Choked up.
	 	0 (Not at all)
	 	1
	 	2
	 	3
	 	4
	 	5
	 	6 (a lot)
3479781 RADIO
	 Qid: 3734715
	 Q: Refreshed, energized, or exhilarated.
	 	0 (Not at all)
	 	1
	 	2
	 	3
	 	4
	 	5
	 	6 (a lot)
3479782 TEXT
3479783 RADIO
	 Qid: 3734716
	 Q: An extraordinary feeling of welcoming or being welcomed.
	 	0 (Not at all)
	 	1
	 	2
	 	3
	 	4
	 	5
	 	6 (a lot)
3479784 RADIO
	 Qid: 3734717
	 Q: An exceptional sense of closeness appear.
	 	0 (Not at all)
	 	1
	 	2
	 	3
	 	4
	 	5
	 	6 (a lot)
3479785 TEXT
3479786 RADIO
	 Qid: 3734718
	 Q: I had positive feelings.
	 	0 (Not a

In [29]:
formID = formes[4]
request_url = 'https://nettskjema.no/api/v2/forms/' + str(formID) + '/submissions?fromSubmissionId=1'
response = session.get(request_url)
subIDs = json.loads(response.content.decode())#eval(response.content.decode())
print(len(subIDs))
if len(subIDs)<5:
    print(subIDs)
else:
    print(subIDs[0])
    

44
{'submissionId': 16662012, 'createdDate': '2021-10-26T00:00:00.000+0200', 'modifiedDate': '2021-10-26T00:00:00.000+0200', 'delivered': True, 'answerTime': 0, 'answers': [{'answerId': 102693987, 'questionId': 3740041, 'answerOptions': [{'answerOptionId': 8635171, 'sequence': 6, 'text': '5', 'preselected': False, 'correct': False}]}, {'answerId': 102694016, 'questionId': 3739808, 'answerOptions': [{'answerOptionId': 8634665, 'sequence': 5, 'text': '5 (Somewhat Agree)', 'preselected': False, 'correct': False}]}, {'answerId': 102694000, 'questionId': 3734724, 'answerOptions': [{'answerOptionId': 8625812, 'sequence': 3, 'text': '2', 'preselected': False, 'correct': False}]}, {'answerId': 102694020, 'questionId': 3740046, 'answerOptions': [{'answerOptionId': 8635205, 'sequence': 5, 'text': '4', 'preselected': False, 'correct': False}]}, {'answerId': 102694018, 'questionId': 3734713, 'answerOptions': [{'answerOptionId': 8625733, 'sequence': 1, 'text': '0 (Not at all)', 'preselected': False

In [ ]:
import pandas

# old notes
Username
Short description
Personal owners
Last active

finnuritmo @ api

finnuritmo @ api

New token created
Token: dar7qsem18ljlqc0ntg8ivbble69nf0g26n8177pk1hlskstanj3gfu78se8q12pro889535p9quqi78akr4hhajijnkj6l0b79aflgdgehlb0ht50870ggkm9t5rnk6fr86
Save the token as it is not stored in the Web form.

The token can be used for the following API calls:
WRITE_FORMS
WRITE_INVITATIONS
READ_SUBMISSIONS
READ_INVITATIONS
READ_FORMS
Allowed IP addresses : 129.240.0.0/16, 193.157.0.0/16

193.157.119.236

https://nettskjema.no/user/api/index.html

https://lcbc-uio.github.io/nettskjemar/articles/auth_setup.html

https://realpython.com/api-integration-in-python/#rest-and-python-consuming-apis

https://docs.python-requests.org/en/latest/user/authentication/



https://nettskjema.no/a/141510


$ curl 'https://nettskjema.no/api/v2/submissions/5136791' -i -X GET \
    -H 'Authorization: Bearer TOKEN'


curl 'https://nettskjema.no/api/v2/users/admin/tokens/expire-date' -i -X GET \ -H 'Authorization: Bearer TOKEN'



$ curl 'https://nettskjema.no/api/v2/submissions/141510' -i -X GET -H 'Authorization: finnuritmo @ api TOKEN'


curl 'https://nettskjema.no/api/v2/users/admin/tokens/expire-date' -i -X GET \ -H 'Authorization: Bearer TOKEN'





*******
base64 --decode customname1.txt > datazip.zip

base64 0c6a885b-de29-a128-1480-fe98e4a06f69.geoLocation.csv  > output.txt

/Desktop/Current_Projects/Resp_Mobile_recording/DataJockying/data(8).zip



